# PROYECTO TEAMSIGHT, CÁLCULO DEL ATTRITION
# (Sección de Machine Learning)

## Adrián Chacón Candia
## BIMEX Analytics

# ESQUEMA SECCIÓ MACHINE LEARNING

1. Fer un Random Forest o Extreme Radiance Boosting per a mirar la dificultat del problema (F1 Score).
    1. Inicialment no fer cap modificació a les dades. Només categoritzar les de resposta múltiple (que no tenen significat de valor) amb Dummy Variables/One-Hot Encoding.
2. Al veure els resultats i mirar F1 Score determinar com seguir.
    1. Si el resultat és bo: 
    2. Si el resultat és dolent:

# COSES A TENIR EN COMPTE

- **CORVES RECALL/ACCURACY**
    - Volem optimitzar el ML (potser no tant alt l'accuracy per a reduïr el nombre de persones)   
    
## COSES A DEMANAR/IDEES PER A LA "SEGONA PART" (FER QÜESTIONARI)
1. Millora de preguntes (general).
2. Preguntes obertes per a poder escriure. Saber si el missatge de la persona és (positiu/neutre/negatiu).

# REWIND ACTUAL

# DUBTES

- Estudio por departamentos
- Que alguien no pese demasiado

# COSES PER A FER

# ÍNDEX

1. [Load the data](#1)
2. [Random Forest](#2)
    1. [One-Hot Encoding](#2.1)

# ÍNDEX UTILITATS
- [llista_categories](#a)
- [dic_responses_of_category](#b)

## 1. Load the data <a class="anchor" id="1"></a>

In [17]:
import pandas as pd

data = pd.read_csv('./WA_Fn-UseC_-HR-Employee-Attrition.csv')
data

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [18]:
#voy a cambiar los valores de ATTRITION: Yes,No a los valores Yes:1, No:0
data_n = data
data_n = data_n.set_index('Attrition')
data_n = data_n.rename({"Yes":1,"No":0})
data_n = data_n.reset_index()
data_n = data_n.set_index('Gender')
data_n = data_n.rename({"Male":1,"Female":-1})
data_n = data_n.reset_index()
data_n = data_n.set_index('OverTime')
data_n = data_n.rename({"Yes":1,"No":0})
data_n = data_n.reset_index()
data_n = data_n.set_index('BusinessTravel')
data_n = data_n.rename({"Non-Travel":0,"Travel_Rarely":0.5,"Travel_Frequently":1})
data_n = data_n.reset_index()
#sí coherente Marital_Status?
data_n = data_n.set_index('MaritalStatus')
data_n = data_n.rename({"Single":0,"Divorced":0.5,"Married":1})
data_n = data_n.reset_index()
#clear nonrelevant data
data_n = data_n.drop(columns=['EmployeeCount','StandardHours','Over18'])

data_n

,MaritalStatus,BusinessTravel,OverTime,Gender,Attrition,Age,DailyRate,Department,DistanceFromHome,Education,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.0,0.5,1,-1,1,41,1102,Sales,1,2,...,3,1,0,8,0,1,6,4,0,5
1,1.0,1.0,0,1,0,49,279,Research & Development,8,1,...,4,4,1,10,3,3,10,7,1,7
2,0.0,0.5,1,1,1,37,1373,Research & Development,2,2,...,3,2,0,7,3,3,0,0,0,0
3,1.0,1.0,1,-1,0,33,1392,Research & Development,3,4,...,3,3,0,8,3,3,8,7,3,0
4,1.0,0.5,0,1,0,27,591,Research & Development,2,1,...,3,4,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,1.0,1.0,0,1,0,36,884,Research & Development,23,2,...,3,3,1,17,3,3,5,2,0,3
1466,1.0,0.5,0,1,0,39,613,Research & Development,6,1,...,3,1,1,9,5,3,7,7,1,7
1467,1.0,0.5,1,1,0,27,155,Research & Development,4,3,...,4,2,1,6,0,3,6,2,0,3
1468,1.0,1.0,0,1,0,49,1023,Sales,2,3,...,3,4,0,17,3,2,9,6,0,8


## 2. Random Forest <a class="anchor" id="2"></a>

### 2.1 One-Hot Encoding <a class="anchor" id="2.1"></a>

Primero vuelvo todo el dataframe numérico volviendo todas las variables de categorías a valores con el *One-Hot Encoding* (si hay respuesta A,B y C hago 3 variables: A Sí/No, etc.).

###### llista_categories <a class="anchor" id="a"></a>

In [19]:
#primero del DataFrame editado miro qué categorías quedan con tipo string.

def llista_categories(DataFrame):
    #función que te da las categorías de un DataFrame en una lista

    llista_categories = [category for category in DataFrame]
    
    return(llista_categories)

In [20]:
lista_cat = llista_categories(data_n)
lista_cat
str_variables = []

for category in lista_cat:
    section = data_n.loc[:,category]
    if type(section[0]) == str:
        str_variables.append(category)
        
str_variables

['Department', 'EducationField', 'JobRole']

###### dic_responses_of_category <a class="anchor" id="b"></a>

In [21]:
def dic_responses_of_category(DataFrame):
    #función que te da en formato de diccionario las posibles respuestas 
    #de cada categoría de un DataFrame dado
    #NECESARIO HABER COMPILADO LA FUNCIÓN: "llista_categories"
    
    list_variables = llista_categories(DataFrame)
    
    possible_answers = {}

    for category in list_variables:
        responses = DataFrame.loc[:,category].value_counts()
        answers = responses.index
        possible_answers[category] = list(answers)

    return(possible_answers)

In [22]:
possible_answers = {}

for category in str_variables:
    responses = data_n.loc[:,category].value_counts()
    answers = responses.index
    possible_answers[category] = list(answers)

possible_answers

{'Department': ['Research & Development', 'Sales', 'Human Resources'],
 'EducationField': ['Life Sciences',
  'Medical',
  'Marketing',
  'Technical Degree',
  'Other',
  'Human Resources'],
 'JobRole': ['Sales Executive',
  'Research Scientist',
  'Laboratory Technician',
  'Manufacturing Director',
  'Healthcare Representative',
  'Manager',
  'Sales Representative',
  'Research Director',
  'Human Resources']}

Ahora que ya tenemos todas las categorías que no tenían respuestas numéricas ya se pueden añadir nuevas variables al DataFrame y volverlas Booleanas.

In [23]:
data_OHE = data_n.loc[:,str_variables]
data_OHE

,Department,EducationField,JobRole
0,Sales,Life Sciences,Sales Executive
1,Research & Development,Life Sciences,Research Scientist
2,Research & Development,Other,Laboratory Technician
3,Research & Development,Life Sciences,Research Scientist
4,Research & Development,Medical,Laboratory Technician
...,...,...,...
1465,Research & Development,Medical,Laboratory Technician
1466,Research & Development,Medical,Healthcare Representative
1467,Research & Development,Life Sciences,Manufacturing Director
1468,Sales,Medical,Sales Executive


In [24]:
from sklearn.preprocessing import OneHotEncoder

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(data_OHE[str_variables]).toarray())

#merge one-hot encoded columns back with original DataFrame
final_df = data_OHE.join(encoder_df)

#view final df
final_df

,Department,EducationField,JobRole,0,1,2,3,4,5,6,...,8,9,10,11,12,13,14,15,16,17
0,Sales,Life Sciences,Sales Executive,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Research & Development,Life Sciences,Research Scientist,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Research & Development,Other,Laboratory Technician,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Research & Development,Life Sciences,Research Scientist,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Research & Development,Medical,Laboratory Technician,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,Research & Development,Medical,Laboratory Technician,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1466,Research & Development,Medical,Healthcare Representative,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1467,Research & Development,Life Sciences,Manufacturing Director,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1468,Sales,Medical,Sales Executive,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [25]:
#drop 'team' column
final_df.drop(str_variables, axis=1, inplace=True)

#rename columns
renaming = []
for category in possible_answers:
    for answer in possible_answers[category]:
        renaming.append(answer)

renaming

final_df.columns = [renaming]

final_df

,Research & Development,Sales,Human Resources,Life Sciences,Medical,Marketing,Technical Degree,Other,Human Resources,Sales Executive,Research Scientist,Laboratory Technician,Manufacturing Director,Healthcare Representative,Manager,Sales Representative,Research Director,Human Resources
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1466,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1467,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1468,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
final_df = final_df.reset_index()
data = data.reset_index()

final_df
data

/Users/Adrian/opt/anaconda3/lib/python3.9/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


,index,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,...,1,80,0,8,0,1,6,4,0,5
1,1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,...,4,80,1,10,3,3,10,7,1,7
2,2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,...,2,80,0,7,3,3,0,0,0,0
3,3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,...,3,80,0,8,3,3,8,7,3,0
4,4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,...,3,80,1,17,3,3,5,2,0,3
1466,1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,...,1,80,1,9,5,3,7,7,1,7
1467,1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,...,2,80,1,6,0,3,6,2,0,3
1468,1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,...,4,80,0,17,3,2,9,6,0,8


In [27]:
data_OHE = pd.merge(data.drop(columns=str_variables), final_df, how='inner',on='index')

final_df = final_df.drop(columns='index')
data = data.drop(columns='index')
data_OHE = data_OHE.drop(columns='index')

data_OHE

ValueError: The column label 'index' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.